In [32]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
heeraldedhia_groceries_dataset_path = kagglehub.dataset_download('heeraldedhia/groceries-dataset')

print('Data source import complete.')


Data source import complete.


In [33]:
# Install it if not installed
!pip install apyori

In [34]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from apyori import apriori

In [35]:
df = pd.read_csv('/content/Groceries_dataset.csv', parse_dates=['Date'])

<ipython-input-35-dd178305dc4d>:1: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [36]:
df.shape

(38765, 3)

In [37]:
df.columns

Index(['Member_number', 'Date', 'itemDescription'], dtype='object')

In [53]:
df.head(10)

,Member_number,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,2015-07-21,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2552,2015-01-05,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2300,2015-09-19,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1187,2015-12-12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3037,2015-02-01,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,4941,2015-02-14,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,4501,2015-05-08,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,3803,2015-12-23,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,2762,2015-03-20,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9,4119,2015-02-12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
df.isnull().any()

,0
Member_number,False
Date,False
itemDescription,False


In [40]:
all_products = df['itemDescription'].unique()
print("Total products: {}".format(len(all_products)))

Total products: 167


In [41]:
df['itemDescription'].value_counts()

,count
itemDescription,
whole milk,2502
other vegetables,1898
rolls/buns,1716
soda,1514
yogurt,1334
...,...
rubbing alcohol,5
bags,4
baby cosmetics,3


In [42]:
df['Member_number'].value_counts()

,count
Member_number,
3180,36
3737,33
3050,33
2051,33
2625,31
...,...
2503,2
3301,2
1775,2


In [43]:
def ditribution_plot(x,y,name=None,xaxis=None,yaxis=None):
    fig = go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(
        title_text=name,
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )
    fig.show()

In [44]:
x = df['itemDescription'].value_counts()
x = x.sort_values(ascending = False)
x = x[:10]

ditribution_plot(x=x.index, y=x.values, yaxis="Count", xaxis="Products")

In [45]:
one_hot = pd.get_dummies(df['itemDescription'])
df.drop('itemDescription', inplace=True, axis=1)
df = df.join(one_hot)
df.head()

,Member_number,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,2015-07-21,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2552,2015-01-05,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2300,2015-09-19,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1187,2015-12-12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3037,2015-02-01,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [46]:
records = df.groupby(["Member_number","Date"])[all_products[:]].apply(sum)
records = records.reset_index()[all_products]

<ipython-input-46-79d526c0d81d>:2: FutureWarning:

The provided callable <built-in function sum> is currently using np.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.sum instead.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



In [47]:
def get_Pnames(x):
    for product in all_products:
        if x[product] > 0:
            x[product] = product
    return x

records = records.apply(get_Pnames, axis=1)
records.head()

<ipython-input-47-5918634529f9>:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'whole milk' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-47-5918634529f9>:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'soda' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-47-5918634529f9>:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'canned beer' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-47-5918634529f9>:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sausage' has dtype incompatible with int64, p

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
0,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
print("Total Transactions: {}".format(len(records)))

Total Transactions: 14963


In [49]:
x = records.values
x = [sub[~(sub == 0)].tolist() for sub in x if sub[sub != 0].tolist()]
transactions = x

In [50]:
transactions[0:10]

[['whole milk', 'pastry', 'salty snack'],
 ['whole milk', 'yogurt', 'sausage', 'semi-finished bread'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['beef', 'white bread'],
 ['frankfurter', 'curd']]

In [51]:
rules = apriori(transactions,min_support=0.00030,min_confidance=0.05,min_lift=3,min_length=2,target="rules")
association_results = list(rules)

In [52]:
for item in association_results:

    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    print("Support: " + str(item[1]))

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: specialty chocolate -> frozen fish
Support: 0.0003341575887188398
Confidence: 0.049019607843137254
Lift: 3.0689556157190907
Rule: liver loaf -> fruit/vegetable juice
Support: 0.00040098910646260775
Confidence: 0.011787819253438114
Lift: 3.52762278978389
Rule: pickled vegetables -> ham
Support: 0.0005346521419501437
Confidence: 0.03125
Lift: 3.4895055970149254
Rule: meat -> roll products 
Support: 0.0003341575887188398
Confidence: 0.019841269841269844
Lift: 3.620547812620984
Rule: misc. beverages -> salt
Support: 0.0003341575887188398
Confidence: 0.0211864406779661
Lift: 3.5619405827461437
Rule: misc. beverages -> spread cheese
Support: 0.0003341575887188398
Confidence: 0.0211864406779661
Lift: 3.170127118644068
Rule: soups -> seasonal products
Support: 0.0003341575887188398
Confidence: 0.04716981132075471
Lift: 14.704205974842766
Rule: spread cheese -> sugar
Support: 0.00040098910646260775
Confidence: 0.06
Lift: 3.3878490566037733
Rule: bottled beer -> sausage
Support: 0.00033415